In [19]:
import yaml 

with open('/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/pair_5_xy_distributed/config.yaml') as f:
    full_config = yaml.load(f, Loader=yaml.FullLoader)

In [23]:
# previous config
# ddpm_config = {
#     'model_params': {
#         'sample_size': 32,
#         'in_channels': 3,
#         'out_channels': 1,
#         'layers_per_block': 2,
#         'block_out_channels': (32, 64, 64),
#         'down_block_types': (
#             "DownBlock2D",
#             "AttnDownBlock2D",
#             "AttnDownBlock2D",
#         ),
#         'up_block_types': (
#             "AttnUpBlock2D",
#             "AttnUpBlock2D",
#             "UpBlock2D",
#         ),
#     },
# } 
 
# ddpm_config = {
#     'model_params': {
#         'sample_size': 32,
#         'in_channels': 3,
#         'out_channels': 1,
#         'layers_per_block': 2,
#         'block_out_channels': (
#             # 64, 64,
#             128, 128,
#             256, 256,
#             # 512, 512
#             ),
#         'down_block_types': (
#             # "DownBlock2D",
#             # "DownBlock2D",
#             "DownBlock2D",
#             "DownBlock2D",
#             "AttnDownBlock2D",
#             "DownBlock2D",
#         ),
#         'up_block_types': (
#             "UpBlock2D",
#             "AttnUpBlock2D",
#             "UpBlock2D",
#             "UpBlock2D",
#             # "UpBlock2D",
#             # "UpBlock2D",
#         ),
#     }
# }

ddpm_config = {
    'model_params': {
        'sample_size': 32,
        'in_channels': 3,
        'out_channels': 1,
        'layers_per_block': 2,
        'block_out_channels': (64, 128, 128, 256),
        'down_block_types': (
            "DownBlock2D",
            "AttnDownBlock2D",
            "AttnDownBlock2D",
            "AttnDownBlock2D",
        ),
        'up_block_types': (
            "AttnUpBlock2D",
            "AttnUpBlock2D",
            "AttnUpBlock2D",
            "UpBlock2D",
        ),
    },
} 

In [24]:
from my_code.models.diag_conditional import DiagConditionedUnet

model = DiagConditionedUnet(ddpm_config["model_params"])

In [ ]:
import numpy as np

# get the number of trainable parameters
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)


old: 1,707,585

full large: 113,670,913

full 4 layers 128 - 256: 28,480,001

32-64-64-128: 4,729,089

64-64-128-128: 7,548,801

# Compare with DiffusionNet

In [13]:
import networks.diffusion_network as diffusion_network

diff_net = diffusion_network.DiffusionNet(
    **full_config['sign_net']['net_params']
    )

In [ ]:
# get the number of trainable parameters
model_parameters = filter(lambda p: p.requires_grad, diff_net.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

In [53]:
from utils.shape_util import read_shape, compute_geodesic_distmat, write_off
import torch
import trimesh

mesh = trimesh.load('/home/s94zalek_hpc/shape_matching/data/FAUST_r/off/tr_reg_008.off')

dist_x = torch.tensor(
                compute_geodesic_distmat(mesh.vertices, mesh.faces)    
            )

dist_x_cdist = torch.cdist(
    torch.tensor(mesh.vertices), 
    torch.tensor(mesh.vertices))

In [ ]:
(dist_x - dist_x_cdist).max()

# UV coordinates

In [64]:
import xatlas

atlas = xatlas.Atlas()

atlas.add_mesh(mesh.vertices, mesh.faces)

# Optionally parametrize the generation with
# `xatlas.ChartOptions` and `xatlas.PackOptions`.
atlas.generate()

vmapping1, indices1, uvs1 = atlas[0]

In [ ]:
uvs1.shape

In [ ]:
vmapping1, indices1, uvs1

In [ ]:
# plot the uv mapping
import matplotlib.pyplot as plt

# plt.scatter(uvs1[:, 0], uvs1[:, 1])

# only show vertices 0-100 from the original mesh


plt.show()

In [ ]:
atlas = xatlas.Atlas()

atlas.add_mesh(mesh1.vertices, mesh1.faces)
atlas.add_mesh(mesh2.vertices, mesh2.faces)

# Optionally parametrize the generation with
# `xatlas.ChartOptions` and `xatlas.PackOptions`.
atlas.generate()

vmapping1, indices1, uvs1 = atlas[0]

In [71]:
# read /home/s94zalek_hpc/shape_matching/figures/texture.png

import PIL


texture = PIL.Image.open('/home/s94zalek_hpc/shape_matching/figures/texture.png')

mesh_unwrapped = mesh.unwrap(texture)

In [ ]:
mesh_unwrapped.show()

In [18]:
import accelerate
import yaml

from my_code.models.diag_conditional import DiagConditionedUnet

# read /home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/pair_5_xy_64_64_128_128/config.yaml
with open('/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/pair_5_xy_64_64_128_128/config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

model = DiagConditionedUnet(config["model_params"])

exp_base_folder = '/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/pair_5_xy_64_64_128_128'
checkpoint_name = 'epoch_99'

if "accelerate" in config and config["accelerate"]:
    accelerate.load_checkpoint_in_model(model, f"{exp_base_folder}/checkpoints/{checkpoint_name}/model.safetensors")